2. 작업형 제2유형

In [1]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata/

fatal: destination path 'yemoonsaBigdata' already exists and is not an empty directory.


In [2]:
import numpy as np
import pandas as pd
X_train = pd.read_csv('./yemoonsaBigdata/datasets/Part2/stellar_X_train.csv')
X_test = pd.read_csv('./yemoonsaBigdata/datasets/Part2/stellar_X_test.csv')
y_train = pd.read_csv('./yemoonsaBigdata/datasets/Part2/stellar_y_train.csv')

In [3]:
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   alpha     70000 non-null  float64
 1   delta     70000 non-null  float64
 2   u         70000 non-null  float64
 3   g         70000 non-null  float64
 4   r         70000 non-null  float64
 5   i         70000 non-null  float64
 6   z         70000 non-null  float64
 7   redshift  70000 non-null  float64
 8   run_ID    70000 non-null  int64  
 9   rerun_ID  70000 non-null  int64  
 10  cam_col   70000 non-null  int64  
 11  field_ID  70000 non-null  int64  
 12  plate     70000 non-null  int64  
dtypes: float64(8), int64(5)
memory usage: 6.9 MB
None


In [4]:
print(X_train.describe())

              alpha         delta             u             g             r  \
count  70000.000000  70000.000000  70000.000000  70000.000000  70000.000000   
mean     177.535809     24.107885     21.938806     20.486718     19.644011   
std       96.483093     19.665057     37.942814     37.925201      1.852053   
min        0.005528    -18.785328  -9999.000000  -9999.000000      9.822070   
25%      127.385304      5.115922     20.349987     18.957325     18.136183   
50%      180.706865     23.579507     22.183860     21.095445     20.122805   
75%      233.942154     39.871336     23.684145     22.121073     21.039012   
max      359.999615     82.947622     32.781390     31.602240     29.571860   

                  i             z      redshift        run_ID  rerun_ID  \
count  70000.000000  70000.000000  70000.000000  70000.000000   70000.0   
mean      19.082969     18.624590      0.575246   4478.649443     301.0   
std        1.755997     37.904705      0.729921   1965.326847  

In [5]:
# 이상치 제거
y_train = y_train.loc[X_train.u != -9999]
X_train = X_train.loc[X_train.u != -9999]

X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

In [6]:
print(X_train.describe())

              alpha         delta             u             g             r  \
count  69999.000000  69999.000000  69999.000000  69999.000000  69999.000000   
mean     177.535145     24.108238     22.081964     20.629856     19.644032   
std       96.483622     19.664976      2.248554      2.033576      1.852057   
min        0.005528    -18.785328     10.996230     10.498200      9.822070   
25%      127.384192      5.117238     20.350000     18.957435     18.136075   
50%      180.705886     23.580630     22.183900     21.095450     20.122810   
75%      233.942330     39.872079     23.684160     22.121075     21.039035   
max      359.999615     82.947622     32.781390     31.602240     29.571860   

                  i             z      redshift        run_ID  rerun_ID  \
count  69999.000000  69999.000000  69999.000000  69999.000000   69999.0   
mean      19.082984     18.767701      0.575254   4478.702681     301.0   
std        1.756005      1.764734      0.729923   1965.290408  

In [7]:
print(X_train['redshift'].quantile([q/100 for q in range(90,101)]))
# 약 2%만이 3을 넘음

0.90    1.566266
0.91    1.675948
0.92    1.786824
0.93    1.910857
0.94    2.041589
0.95    2.186017
0.96    2.323494
0.97    2.485796
0.98    2.713839
0.99    3.116165
1.00    7.011245
Name: redshift, dtype: float64


In [8]:
# 파생 변수 추가
X_train['redshift_upper3'] = np.where(X_train['redshift'] > 3,1,0)
X_test['redshift_upper3'] = np.where(X_test['redshift'] > 3,1,0)

In [9]:
col_del = []
col_num = ['alpha','delta','u','g','r','i','z','redshift']
col_cat = ['run_ID','rerun_ID','cam_col','field_ID','plate', 'redshift_upper3']
col_y = ['galaxy']

X_train[col_cat] = X_train[col_cat].astype(str)
X_test[col_cat] = X_test[col_cat].astype(str)

X_train = X_train.drop(col_del, axis=1)
X_test = X_test.drop(col_del, axis=1)

In [10]:
train_df = pd.concat([X_train, y_train], axis=1)

for _col in col_num:
    print('-'*80)
    print(_col)
    print(train_df.groupby(col_y)[_col].describe(),end='\n\n')
    
# 수치형 중 의미있어 보이는 변수 redshift

--------------------------------------------------------------------------------
alpha
          count        mean        std       min         25%         50%  \
galaxy                                                                     
0       28433.0  177.234737  99.202036  0.010959  120.997187  180.253146   
1       41566.0  177.740638  94.579769  0.005528  131.392267  181.032298   

               75%         max  
galaxy                          
0       239.025694  359.999615  
1       231.397544  359.989386  

--------------------------------------------------------------------------------
delta
          count       mean        std        min       25%        50%  \
galaxy                                                                  
0       28433.0  25.067356  20.141007 -18.785328  5.060842  26.372836   
1       41566.0  23.452158  19.305416 -12.364701  5.136824  22.255904   

              75%        max  
galaxy                        
0       41.762988  82.947622  
1 

In [11]:
for _col in col_cat:
    print(train_df.groupby(_col, as_index=False)[col_y].mean().sort_values(by=col_y, ascending=False))
    # 범주형 변수 중 의미있어 보이는 변수 : cam_col, redshift_upper3

    run_ID  galaxy
97    2883     1.0
325   6121     1.0
399   8054     1.0
38    1897     1.0
245   4800     1.0
..     ...     ...
292   5378     0.0
279   5237     0.0
268   5115     0.0
261   5060     0.0
424    994     0.0

[425 rows x 2 columns]
  rerun_ID    galaxy
0      301  0.593808
  cam_col    galaxy
0       1  0.619151
3       4  0.612120
4       5  0.610490
2       3  0.593031
1       2  0.583152
5       6  0.530405
    field_ID  galaxy
676      717     1.0
750      788     1.0
667      708     1.0
704      746     1.0
706      748     1.0
..       ...     ...
830      934     0.0
829      933     0.0
828      932     0.0
827      931     0.0
835      940     0.0

[847 rows x 2 columns]
     plate  galaxy
6186   999     1.0
4292  6417     1.0
4263  6388     1.0
4264  6389     1.0
1082  1900     1.0
...    ...     ...
1653  2560     0.0
1655  2563     0.0
1656  2564     0.0
3569  5343     0.0
1073  1891     0.0

[6187 rows x 2 columns]
  redshift_upper3    galaxy
0        

In [12]:
from sklearn.preprocessing import LabelEncoder
X = pd.concat([X_train, X_test])

for _col in col_cat:
    le = LabelEncoder()
    le.fit(X[_col])
    X_train[_col] = le.transform(X_train[_col])
    X_test[_col] = le.transform(X_test[_col])

In [13]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train)

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_tr[col_num] = scaler.fit_transform(X_tr[col_num])
X_val[col_num] = scaler.transform(X_val[col_num])
X_test[col_num] = scaler.transform(X_test[col_num])

C:\Users\lg\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
C:\Users\lg\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [16]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()
model_rf.fit(X_tr, y_tr.values.ravel())

RandomForestClassifier()

In [18]:
from xgboost import XGBClassifier

model_xgb1 = XGBClassifier()
model_xgb1.fit(X_tr, y_tr.values.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [19]:
from sklearn.metrics import roc_auc_score

y_pred_rf = model_rf.predict_proba(X_val)
y_pred_xgb1 = model_xgb1.predict_proba(X_val)

score_rf = roc_auc_score(y_val, y_pred_rf[:,1])
score_xgb1 = roc_auc_score(y_val, y_pred_xgb1[:,1])

print(score_rf, score_xgb1)

0.9907650483082023 0.9890822757736975


In [21]:
print(pd.DataFrame({'feature' : X_tr.columns, 'fi_rf' : model_rf.feature_importances_, 
                    'fi_xgb' : model_xgb1.feature_importances_}))

            feature     fi_rf    fi_xgb
0             alpha  0.015815  0.009571
1             delta  0.014780  0.011165
2                 u  0.076251  0.024073
3                 g  0.080660  0.090870
4                 r  0.051962  0.016085
5                 i  0.063126  0.021588
6                 z  0.114303  0.039535
7          redshift  0.502019  0.739468
8            run_ID  0.010959  0.009906
9          rerun_ID  0.000000  0.000000
10          cam_col  0.003861  0.010260
11         field_ID  0.008368  0.008461
12            plate  0.048993  0.019018
13  redshift_upper3  0.008904  0.000000


In [25]:
pred = model_xgb1.predict_proba(X_test)[:,1]
pd.DataFrame({'index' : X_test.index, 'target' : pred}).to_csv('./yemoonsaBigdata/res/003000000.csv')